In [6]:
#similar word extraction on the basis of word 

In [7]:
#!pip install tweepy

In [10]:
import os
import tweepy as tw
import pandas as pd
import re, string
from tqdm import tqdm
from sklearn.decomposition import PCA 

In [11]:
consumer_key= 'mxtkW7a8EYHwBd3rMBmWrscda'
consumer_secret= 'qMGuky46B5AJddQ8bO4DfrQnUHuaQ18kVa8k7eQE3zBhtr6Jb4'
access_token= '3267248131-B27E5BJdErwnN1kDUxTSvDrToYVUnZkbfbkiwmp'
access_token_secret= 'jpYfRJ0usnPVg8QurTQiiNzNLjimOobetHNCHT7q3btLc'

In [12]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [41]:
# Define the search term and the date_since date as variables
search_words = "@Citi"
date_since = "2019-11-01"

In [42]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(3000)


In [43]:
data=[]
for s in tqdm(tweets):
    data.append(s.text.encode('utf8') )

2680it [01:10, 50.48it/s]

KeyboardInterrupt: 

In [44]:
len(data)

2694

In [45]:
alltwt=[]
for i in data:
    x =str(i, 'utf-8')
    alltwt.append(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))

<>:4: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
<ipython-input-45-c524f21fe8d4>:4: DeprecationWarning: invalid escape sequence \w
  alltwt.append(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))


In [46]:
#remove the stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

In [47]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
out_arr=[]
for twt in tqdm(alltwt):
    data = twt.split()
    inner_arr=[]
    for i in data:
        if i not in stop_words:
            inner_arr.append((lemmatizer.lemmatize(i.lower())))
    out_arr.append(inner_arr)


100%|██████████| 2694/2694 [00:00<00:00, 13968.97it/s]


In [48]:
from gensim.models import Word2Vec
import gensim

In [49]:
model = gensim.models.Word2Vec(out_arr, min_count=1,size=200,window=2,iter=20,workers=2)

In [50]:
word = "service"
model.wv.most_similar(positive=word, topn=20)

[('financial', 0.917086124420166),
 ('customer', 0.9163394570350647),
 ('vacation', 0.9028521776199341),
 ('hello', 0.8977997303009033),
 ('within', 0.8952398300170898),
 ('miserable', 0.8945362567901611),
 ('regtech', 0.892692506313324),
 ('direct', 0.8887707591056824),
 ('approach', 0.8690682649612427),
 ('challenging', 0.8678646683692932),
 ('excellent', 0.8658445477485657),
 ('called', 0.8631377220153809),
 ('breach', 0.8630833625793457),
 ('innovating', 0.8630340099334717),
 ('internet', 0.8554661273956299),
 ('cambodia', 0.8533862233161926),
 ('money', 0.8519916534423828),
 ('leave', 0.850392758846283),
 ('vrsec', 0.8494607210159302),
 ('allowed', 0.849337100982666)]

In [51]:
word = "customer"
model.wv.most_similar(positive=word, topn=20)

[('hello', 0.972855269908905),
 ('15', 0.9566177129745483),
 ('disappointed', 0.9556820392608643),
 ('reply', 0.9554550647735596),
 ('experience', 0.9549350738525391),
 ('approach', 0.9543461203575134),
 ('vrsec', 0.9538570642471313),
 ('vacation', 0.9538004398345947),
 ('within', 0.9506697654724121),
 ('challenging', 0.9504146575927734),
 ('none', 0.9492953419685364),
 ('original', 0.9488800168037415),
 ('experienced', 0.9486130475997925),
 ('sooooo', 0.9480211734771729),
 ('excellent', 0.9478511810302734),
 ('accidentally', 0.9461226463317871),
 ('miserable', 0.9459228515625),
 ('heard', 0.9452987909317017),
 ('cambodia', 0.9452584981918335),
 ('economicgrowth', 0.9450675845146179)]

In [52]:
model.wv.similarity(w1="good", w2="bad")

0.9299686

In [53]:
model.wv.evaluate_word_analogies('questions-words.txt', restrict_vocab=20000, case_insensitive=True)

/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)


(0.0,
 [{'section': 'capital-common-countries',
   'correct': [],
   'incorrect': [('BEIJING', 'CHINA', 'LONDON', 'ENGLAND'),
    ('LONDON', 'ENGLAND', 'BEIJING', 'CHINA')]},
  {'section': 'capital-world', 'correct': [], 'incorrect': []},
  {'section': 'currency',
   'correct': [],
   'incorrect': [('BRAZIL', 'REAL', 'USA', 'DOLLAR'),
    ('USA', 'DOLLAR', 'BRAZIL', 'REAL')]},
  {'section': 'city-in-state', 'correct': [], 'incorrect': []},
  {'section': 'family',
   'correct': [],
   'incorrect': [('FATHER', 'MOTHER', 'HE', 'SHE'),
    ('FATHER', 'MOTHER', 'MAN', 'WOMAN'),
    ('HE', 'SHE', 'MAN', 'WOMAN'),
    ('HE', 'SHE', 'FATHER', 'MOTHER'),
    ('MAN', 'WOMAN', 'FATHER', 'MOTHER'),
    ('MAN', 'WOMAN', 'HE', 'SHE')]},
  {'section': 'gram1-adjective-to-adverb', 'correct': [], 'incorrect': []},
  {'section': 'gram2-opposite', 'correct': [], 'incorrect': []},
  {'section': 'gram3-comparative',
   'correct': [],
   'incorrect': [('BAD', 'WORSE', 'BIG', 'BIGGER'),
    ('BAD', 'WORSE', 

<h1>Topic Modeling LDA</h1>

In [55]:
from gensim import corpora
dictionary = corpora.Dictionary(out_arr)
corpus = [dictionary.doc2bow(text) for text in out_arr]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)


In [56]:
import gensim
NUM_TOPICS = 4
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=6)
for topic in topics:
    print(topic)

(0, '0.083*"rt" + 0.023*"demand" + 0.021*"latest" + 0.021*"series" + 0.020*"concert" + 0.020*"127"')
(1, '0.043*"rt" + 0.028*"amp" + 0.027*"citi" + 0.026*"head" + 0.026*"global" + 0.025*"sara"')
(2, '0.079*"rt" + 0.072*"time" + 0.070*"next" + 0.069*"presented" + 0.069*"100" + 0.069*"honoree"')
(3, '0.036*"rt" + 0.025*"proud" + 0.022*"year" + 0.019*"he" + 0.019*"named" + 0.019*"meet"')


/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)


In [57]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)


/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
/Users/mohitdedhe/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args,

<h4>TFIDF</h4>

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(alltwt)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [60]:
df.to_csv('tfidf.csv')